In [1]:
from bs4 import BeautifulSoup
import lxml
import html5lib
import re
import json
import uuid
import glob
import pickle
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.cluster import KMeansClusterer, euclidean_distance
from gensim import corpora, models, utils
from numpy import array
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from pprint import pprint
from gensim import corpora
import re
from os.path import basename
import string
import logging
import pandas as pd
import numpy as np

In [12]:
Topics_Name=["Topic 1", "Topic 2", "Topic 3", "Topic 4", "Topic 5", "Topic 6","Topic 7",\
           "Topic 8","Topic 9","Topic 10","Topic 11","Topic 12","Topic 13","Topic 14","Topic 15",\
           "Topic 16","Topic 17","Topic 18","Topic 19","Topic 20","Topic 21","Topic 22","Topic 23",\
           "Topic 24","Topic 25","Topic 26","Topic 27","Topic 28","Topic 29","Topic 30","Topic 31",\
           "Topic 32","Topic 33","Topic 34","Topic 35","Topic 36","Topic 37","Topic 38","Topic 39","Topic 40",\
           "Topic 41","Topic 42","Topic 43","Topic 44","Topic 45","Topic 46","Topic 47","Topic 48","Topic 49","Topic 50"]

In [2]:
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.max_columns", 100)

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [4]:
Mommit = pd.read_pickle('MommitReddit.pkl')

In [5]:
len(Mommit.index)

203036

In [14]:
def find_intersection(TextList,LDAList):
    intersection = list(set(TextList) & set(LDAList))
    return len(intersection)

In [13]:
def clean_LDA_topics(LDATopics):
    addch = "'"
    topics = []
    for i in range(0,len(LDATopics)):
        current_topic  = LDATopics[i][1]
        print(current_topic)
        newwords = []
        split_current_topic = []
        split_current_topic = current_topic.split("+")
        for j in range(0, len(split_current_topic)):
            newword = split_current_topic[j]
            print(split_current_topic)
            if j==0:
                newword = newword[7:-2]
            else:
                newword = newword[8:-2]
                newword = newword[:0] + newword[0:]
            newwords.append(newword)
        topics.append(newwords)
    return topics

In [6]:
def Tokinization(document):
    document = "".join(document)
    tokenizer = RegexpTokenizer(r'\w+')
    intermediate = tokenizer.tokenize(document)
    #stop words
    stop = stopwords.words('english')
    intermediate = [i for i in intermediate if i not in stop]
    #stemming
    lanste = LancasterStemmer()
    intermediate = [lanste.stem(i) for i in intermediate]
    return intermediate

In [7]:
def Preprocess(document):
    #document = document.encode('ascii','ignore')
    #remove http
    intermediate = document
    #join
    intermediate = "".join(document)
    intermediate = re.sub(r'htt\S+', '', intermediate)
    intermediate = re.sub(r'://t.co\S*', '', intermediate)
    # remove rts
    intermediate = re.sub(r'\brt\b','', intermediate)
    # remove digits
    intermediate = re.sub(r'\d+','', intermediate)
    # remove names after @
    # (screen names)
    intermediate = re.sub("@\\w+ *", '', intermediate)
    intermediate = re.sub(r'\b@\b','', intermediate)
    intermediate = unescape(intermediate)
    exclude = set(string.punctuation)
    intermediate = "".join(ch for ch in intermediate if ch is not exclude)
    
    intermediate = Tokinization(intermediate)
    return intermediate

In [8]:
def removeNone(intermediate):
    #remove None Values
    intermediate = ['' if v is None else v for v in intermediate]
    return intermediate

In [9]:
def unescape(s):
    s = s.replace("&gt;", "greater than")
    s = s.replace("&lt;3", "love")
    s = s.replace("&lt;", "less than")
    s = s.replace("&amp;", "and")
    s = s.replace("via", '')
    s = s.replace('\\n', '')
    s = s.replace('\\t', '')
    s = s.replace('\\r', '')
    return s

In [18]:
Mommit['body'] = Mommit['body'].str.lower()
bodylist = Mommit['body'].tolist()
NoneRemovedbodylist = removeNone(bodylist)
Mommit['body']=NoneRemovedbodylist
CompleteThread = Mommit.groupby('link_id')['body'].apply(list)

In [19]:
processed_posts = []
#creating threads for each of the users
for post in NoneRemovedbodylist:
    #Preprocessing each of the threads
    processed_posts.append(Preprocess(post))

In [11]:
len(CompleteThread)

13649

In [81]:
len(Mommit.author.unique())

15732

In [11]:
#running for the rest of the data
processed_threads = []
#creating threads for each of the users
for thread in CompleteThread:
    #Preprocessing each of the threads
    processed_threads.append(Preprocess(thread))

In [12]:
texts = processed_threads
texts[4]

['nommmh',
 'look',
 'lik',
 'heav',
 'wish',
 'could',
 'eat',
 'cak',
 'abandon',
 'manufact',
 'mess',
 'cut',
 'pic',
 'lik',
 'son',
 'want',
 'get',
 'ic',
 'fac',
 'stil',
 'lik',
 'sticky',
 'wet',
 'thing',
 'anywh',
 'near',
 'fac',
 'lol',
 'thank',
 'lov',
 'mom',
 'post',
 'origin',
 'daddit',
 'quit',
 'get',
 'amaz',
 'mileston',
 'wif',
 'perfect',
 'healthy',
 'baby',
 'boy',
 'enjoy',
 'person',
 'smash',
 'cak',
 'deserv',
 'best',
 'famy',
 'near',
 'end',
 'see',
 'think',
 'sug',
 'rush',
 'start',
 'get',
 'growl',
 'mou',
 'op',
 'squ',
 'cak',
 'fing',
 'uns',
 'ic',
 'first',
 'ev',
 'got',
 'good',
 'stuff',
 'went',
 'crazy',
 'yeah',
 'son',
 'dainty',
 'pick',
 'fond',
 'dot',
 'at',
 'on',
 'on',
 'gag',
 'barf',
 'end',
 'smash',
 'cak',
 'min',
 'ding',
 'next',
 'mon',
 'termy',
 'premmy',
 'storm',
 'thing',
 'asham',
 'say',
 'pict',
 'mad',
 'cry',
 'sil',
 'mad',
 'request',
 'lov',
 'tier',
 'elmo',
 'cak',
 'gav',
 'boyth',
 'slic',
 'look',
 'li

In [13]:
# Get token frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token]+=1

In [14]:
tokenized = [[token for token in text if frequency[token]>1]for text in texts]

In [15]:
wordfreq = {}
for text in texts:
    for token in text:
        wordfreq[token]=frequency[token]

In [16]:
higherfreq2 = {k:v for k, v in wordfreq.items() if v>10000}  
higherfreq2

{'act': 15146,
 'also': 28114,
 'alway': 12702,
 'around': 14736,
 'ask': 11622,
 'baby': 54986,
 'back': 18235,
 'bed': 10669,
 'best': 10990,
 'bet': 14344,
 'bit': 10017,
 'car': 16817,
 'chang': 10417,
 'child': 19068,
 'com': 17697,
 'could': 19343,
 'daught': 19100,
 'day': 30285,
 'diff': 11748,
 'easy': 11301,
 'eat': 11589,
 'ev': 32510,
 'every': 13984,
 'famy': 11236,
 'feel': 26750,
 'fin': 11138,
 'find': 10970,
 'first': 22337,
 'friend': 11917,
 'get': 67775,
 'giv': 18367,
 'go': 25030,
 'going': 19993,
 'good': 25499,
 'got': 19400,
 'gre': 16744,
 'hard': 12307,
 'help': 27588,
 'hom': 13293,
 'hour': 12951,
 'husband': 15821,
 'keep': 13302,
 'kid': 39627,
 'know': 31653,
 'let': 14814,
 'lik': 58435,
 'littl': 22916,
 'long': 14192,
 'look': 16671,
 'lot': 18555,
 'lov': 21369,
 'mad': 10600,
 'mak': 32837,
 'mayb': 11073,
 'mom': 22092,
 'month': 25659,
 'much': 25587,
 'nee': 28192,
 'nev': 15867,
 'night': 15278,
 'nurs': 10863,
 'old': 28788,
 'on': 45602,
 'par

In [17]:
import operator
x = higherfreq2
sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x

[('bit', 10017),
 ('chang', 10417),
 ('play', 10551),
 ('mad', 10600),
 ('bed', 10669),
 ('nurs', 10863),
 ('talk', 10895),
 ('find', 10970),
 ('best', 10990),
 ('said', 11012),
 ('mayb', 11073),
 ('tel', 11121),
 ('fin', 11138),
 ('famy', 11236),
 ('easy', 11301),
 ('eat', 11589),
 ('ask', 11622),
 ('diff', 11748),
 ('friend', 11917),
 ('though', 11979),
 ('hard', 12307),
 ('alway', 12702),
 ('pregn', 12749),
 ('hour', 12951),
 ('hom', 13293),
 ('keep', 13302),
 ('every', 13984),
 ('long', 14192),
 ('sur', 14254),
 ('two', 14305),
 ('bet', 14344),
 ('around', 14736),
 ('let', 14814),
 ('right', 14927),
 ('act', 15146),
 ('night', 15278),
 ('husband', 15821),
 ('nev', 15867),
 ('put', 16589),
 ('look', 16671),
 ('gre', 16744),
 ('someth', 16771),
 ('car', 16817),
 ('see', 16840),
 ('wel', 16893),
 ('par', 16912),
 ('thank', 17207),
 ('peopl', 17637),
 ('com', 17697),
 ('back', 18235),
 ('giv', 18367),
 ('lot', 18555),
 ('child', 19068),
 ('daught', 19100),
 ('could', 19343),
 ('got', 1

In [18]:
dictionary = corpora.Dictionary(texts)
dictionary.save('Mommitdict.dict')
print(dictionary)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(31589 unique tokens: ['childless', 'cho', 'year', 'stil', 'espec']...)
INFO:gensim.corpora.dictionary:built Dictionary(39256 unique tokens: ['childless', 'cho', 'year', 'stil', 'espec']...) from 13649 documents (total 5718951 corpus positions)
INFO:gensim.utils:saving Dictionary object under Mommitdict.dict, separately None
INFO:gensim.utils:saved Mommitdict.dict


Dictionary(39256 unique tokens: ['childless', 'cho', 'year', 'stil', 'espec']...)


In [19]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('MommitCorpus.mm', corpus)

INFO:gensim.corpora.mmcorpus:storing corpus in Matrix Market format to MommitCorpus.mm
INFO:gensim.matutils:saving sparse matrix to MommitCorpus.mm
INFO:gensim.matutils:PROGRESS: saving document #0
INFO:gensim.matutils:PROGRESS: saving document #1000
INFO:gensim.matutils:PROGRESS: saving document #2000
INFO:gensim.matutils:PROGRESS: saving document #3000
INFO:gensim.matutils:PROGRESS: saving document #4000
INFO:gensim.matutils:PROGRESS: saving document #5000
INFO:gensim.matutils:PROGRESS: saving document #6000
INFO:gensim.matutils:PROGRESS: saving document #7000
INFO:gensim.matutils:PROGRESS: saving document #8000
INFO:gensim.matutils:PROGRESS: saving document #9000
INFO:gensim.matutils:PROGRESS: saving document #10000
INFO:gensim.matutils:PROGRESS: saving document #11000
INFO:gensim.matutils:PROGRESS: saving document #12000
INFO:gensim.matutils:PROGRESS: saving document #13000
INFO:gensim.matutils:saved 13649x39256 matrix, density=0.478% (2561787/535805144)
DEBUG:gensim.matutils:closi

In [31]:
MommitLDA = models.ldamodel.LdaModel
ldamodelMommit = MommitLDA(corpus, num_topics=30, id2word=dictionary, passes=20)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.03333333333333333
INFO:gensim.models.ldamodel:using symmetric eta at 2.5473812920317912e-05
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 30 topics, 20 passes over the supplied corpus of 13649 documents, updating model once every 2000 documents, evaluating perplexity every 13649 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #2000/13649
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 2000 documents
DEBUG:gensim.models.ldamodel:338/2000 documents converged within 50 iterations
DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 13649 documents
INFO:gensim.models.ldamodel:topic #29 (0.033): 0.010*"lik" + 0.010*"get" + 0.010*"on" + 0.009*"baby" + 0.009*"us" + 0.008*"tim" + 0.008*"real" + 0.007*"e

In [22]:
ldamodelMommit.print_topics(num_topics=50, num_words=30)

INFO:gensim.models.ldamodel:topic #0 (0.020): 0.021*"school" + 0.020*"teach" + 0.014*"son" + 0.013*"word" + 0.013*"kid" + 0.012*"start" + 0.012*"talk" + 0.011*"learn" + 0.011*"ear" + 0.011*"us" + 0.010*"get" + 0.010*"sign" + 0.009*"help" + 0.009*"year" + 0.008*"develop" + 0.008*"say" + 0.008*"month" + 0.008*"speech" + 0.008*"class" + 0.008*"child" + 0.007*"lik" + 0.007*"langu" + 0.007*"thing" + 0.007*"know" + 0.007*"daught" + 0.006*"real" + 0.006*"ag" + 0.006*"aut" + 0.006*"diff" + 0.006*"therapy"
INFO:gensim.models.ldamodel:topic #1 (0.020): 0.031*"bir" + 0.022*"hospit" + 0.022*"lab" + 0.020*"baby" + 0.019*"pain" + 0.014*"hour" + 0.013*"push" + 0.013*"epid" + 0.012*"contract" + 0.010*"want" + 0.009*"induc" + 0.009*"nat" + 0.009*"midw" + 0.009*"get" + 0.008*"lik" + 0.008*"real" + 0.008*"expery" + 0.007*"tim" + 0.007*"nurs" + 0.007*"birth" + 0.007*"hom" + 0.006*"would" + 0.006*"delivery" + 0.006*"on" + 0.006*"giv" + 0.006*"got" + 0.006*"ev" + 0.006*"thing" + 0.006*"wat" + 0.006*"first"


[(0,
  '0.021*"school" + 0.020*"teach" + 0.014*"son" + 0.013*"word" + 0.013*"kid" + 0.012*"start" + 0.012*"talk" + 0.011*"learn" + 0.011*"ear" + 0.011*"us" + 0.010*"get" + 0.010*"sign" + 0.009*"help" + 0.009*"year" + 0.008*"develop" + 0.008*"say" + 0.008*"month" + 0.008*"speech" + 0.008*"class" + 0.008*"child" + 0.007*"lik" + 0.007*"langu" + 0.007*"thing" + 0.007*"know" + 0.007*"daught" + 0.006*"real" + 0.006*"ag" + 0.006*"aut" + 0.006*"diff" + 0.006*"therapy"'),
 (1,
  '0.031*"bir" + 0.022*"hospit" + 0.022*"lab" + 0.020*"baby" + 0.019*"pain" + 0.014*"hour" + 0.013*"push" + 0.013*"epid" + 0.012*"contract" + 0.010*"want" + 0.009*"induc" + 0.009*"nat" + 0.009*"midw" + 0.009*"get" + 0.008*"lik" + 0.008*"real" + 0.008*"expery" + 0.007*"tim" + 0.007*"nurs" + 0.007*"birth" + 0.007*"hom" + 0.006*"would" + 0.006*"delivery" + 0.006*"on" + 0.006*"giv" + 0.006*"got" + 0.006*"ev" + 0.006*"thing" + 0.006*"wat" + 0.006*"first"'),
 (2,
  '0.114*"book" + 0.075*"read" + 0.022*"phon" + 0.021*"ap" + 0.01

In [24]:
ldamodelMommit.save('MommitLDA.model')

INFO:gensim.utils:saving LdaState object under MommitLDA.model.state, separately None
INFO:gensim.utils:saved MommitLDA.model.state
INFO:gensim.utils:saving LdaModel object under MommitLDA.model, separately ['expElogbeta', 'sstats']
INFO:gensim.utils:storing np array 'expElogbeta' to MommitLDA.model.expElogbeta.npy
INFO:gensim.utils:not storing attribute dispatcher
INFO:gensim.utils:not storing attribute id2word
INFO:gensim.utils:not storing attribute state
INFO:gensim.utils:saved MommitLDA.model


In [15]:
MommitLDA = models.ldamodel.LdaModel
ldamodelmommit = MommitLDA.load('MommitLDA.model')
LDA_topics= ldamodelmommit.print_topics(num_topics=50, num_words=30)
All_topics = clean_LDA_topics(LDA_topics)

INFO:gensim.utils:loading LdaModel object from MommitLDA.model
INFO:gensim.utils:loading expElogbeta from MommitLDA.model.expElogbeta.npy with mmap=None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:setting ignored attribute id2word to None
INFO:gensim.utils:setting ignored attribute state to None
INFO:gensim.utils:loaded MommitLDA.model
INFO:gensim.utils:loading LdaModel object from MommitLDA.model.state
INFO:gensim.utils:loaded MommitLDA.model.state
INFO:gensim.models.ldamodel:topic #0 (0.020): 0.021*"school" + 0.020*"teach" + 0.014*"son" + 0.013*"word" + 0.013*"kid" + 0.012*"start" + 0.012*"talk" + 0.011*"learn" + 0.011*"ear" + 0.011*"us" + 0.010*"get" + 0.010*"sign" + 0.009*"help" + 0.009*"year" + 0.008*"develop" + 0.008*"say" + 0.008*"month" + 0.008*"speech" + 0.008*"class" + 0.008*"child" + 0.007*"lik" + 0.007*"langu" + 0.007*"thing" + 0.007*"know" + 0.007*"daught" + 0.006*"real" + 0.006*"ag" + 0.006*"aut" + 0.006*"diff" + 0.006*"therapy"
INFO:g

0.021*"school" + 0.020*"teach" + 0.014*"son" + 0.013*"word" + 0.013*"kid" + 0.012*"start" + 0.012*"talk" + 0.011*"learn" + 0.011*"ear" + 0.011*"us" + 0.010*"get" + 0.010*"sign" + 0.009*"help" + 0.009*"year" + 0.008*"develop" + 0.008*"say" + 0.008*"month" + 0.008*"speech" + 0.008*"class" + 0.008*"child" + 0.007*"lik" + 0.007*"langu" + 0.007*"thing" + 0.007*"know" + 0.007*"daught" + 0.006*"real" + 0.006*"ag" + 0.006*"aut" + 0.006*"diff" + 0.006*"therapy"
['0.021*"school" ', ' 0.020*"teach" ', ' 0.014*"son" ', ' 0.013*"word" ', ' 0.013*"kid" ', ' 0.012*"start" ', ' 0.012*"talk" ', ' 0.011*"learn" ', ' 0.011*"ear" ', ' 0.011*"us" ', ' 0.010*"get" ', ' 0.010*"sign" ', ' 0.009*"help" ', ' 0.009*"year" ', ' 0.008*"develop" ', ' 0.008*"say" ', ' 0.008*"month" ', ' 0.008*"speech" ', ' 0.008*"class" ', ' 0.008*"child" ', ' 0.007*"lik" ', ' 0.007*"langu" ', ' 0.007*"thing" ', ' 0.007*"know" ', ' 0.007*"daught" ', ' 0.006*"real" ', ' 0.006*"ag" ', ' 0.006*"aut" ', ' 0.006*"diff" ', ' 0.006*"therap

In [16]:
topiclist = All_topics

In [63]:
topiclist[9]

['us',
 'skin',
 'wash',
 'work',
 'oil',
 'cream',
 'dry',
 'rash',
 'also',
 'help',
 'wat',
 'smel',
 'get',
 'lot',
 'baby',
 'soap',
 'lik',
 'mak',
 'try',
 'bath',
 'clear',
 'good',
 'cle',
 'put',
 'deterg',
 'produc',
 'wel',
 'wip',
 'real',
 'stuf']

In [20]:
currentList = []
mega_topic_list= [[] for _ in range(len(topiclist)) ]
for i in range(0, len(mega_topic_list)):
    currentList = mega_topic_list[i]
    currentTopic = topiclist[i]
    for j in range (0, len(processed_posts)):
        intersected = find_intersection(processed_posts[j], currentTopic)
        currentList.append(intersected)

In [21]:
for i in range (0,len(Topics_Name)):
    Mommit[Topics_Name[i]]= mega_topic_list[i]

In [25]:
#Create Mommit Word2Vec model
from gensim.models import Word2Vec
mommitmodel = Word2Vec(texts, size=100, window=5, min_count=5, workers=4)

DEBUG:gensim.models.word2vec:Fast version of gensim.models.word2vec is being used
INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 3794850 words, keeping 31589 word types
INFO:gensim.models.word2vec:collected 39256 word types from a corpus of 5718951 raw words and 13649 sentences
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:min_count=5 retains 12842 unique words (32% of original 39256, drops 26414)
INFO:gensim.models.word2vec:min_count=5 leaves 5678995 word corpus (99% of original 5718951, drops 39956)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 39256 items
INFO:gensim.models.word2vec:sample=0.001 downsamples 64 most-common words
INFO:gensim.models.word2vec:downsampling leaves estimated 5129064 word corpus (90.3% of prior 5678995)
INFO:gensim.models.w

In [26]:
mommitmodel.save('Mommitword2vec.model')

INFO:gensim.utils:saving Word2Vec object under Mommitword2vec.model, separately None
INFO:gensim.utils:not storing attribute syn0norm
INFO:gensim.utils:not storing attribute cum_table
INFO:gensim.utils:saved Mommitword2vec.model
